In [1]:
import pandas as pd
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt
import utils
from utils import Plane, Handler
import visualize
import pickle

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Trabajo previo - Contexto y relevamiento de información

- En base al relevamiento de información realizado, observamos que la cantidad de vuelos que llegan y salen de Aeroparque son 300. Suponiendo que salen y llegan la misma cantidad de aviones, eso nos da un total de 150 aviones por día. Dado que nuestra simulación contempla solamente 18 horas del día, la cantidad de aviones que llegan a AEP por hora esta dada por la cuenta $\frac{150}{24} = 6.25 \approx 6$. Por ende, la tasa de arribos $\lambda = 6/60 = 0.1$  
- Suponiendo que no hay congestión, y el avión puede viajar a la velocidad máxima permitida en cada tramo, el tiempo que tarda es el siguiente: $(\frac{100 - 50}{300} + \frac{50 - 15}{250} + \frac{15 - 5}{200} + \frac{5 - 0}{150}) \cdot 60 \approx 23 \ minutos$
- El aeropuerto de Rosario se encuentra a un total de 154 millas náuticas de AEP (apróximadamente 286km), por ende, siguiendo con la misma idea que en el inciso anterios: $(\frac{154 - 100}{500} + \frac{100 - 50}{300} + \frac{50 - 15}{250} + \frac{15 - 5}{200} + \frac{5 - 0}{150}) \cdot 60 \approx 30 \ minutos$. Esto no es consistente con los horarios publicados por Aerolíneas (1 hora de viaje aproximadamente). Sin embargo, esto puede deberse a que en nuestro cálculo no estamos considerando variables como el tiempo de despegue y aterrizaje, tiempo de aceleración y desaceleración, congestión, etc. Lo mismo aplica para el inciso anterior; los tiempos y valores se volverán más realistas a medida que realicemos las simulaciones.
- La máxima cantidad de aterrizajes que podrían suceder entre las 6 am y la medianoche está dada por la siguiente cuenta: $\frac{18 \cdot 60}{4}= 270$. Entonces, la cantidad máxima de aterrizajes en 18 horas suponiendo la restricción mencionada anteriormente es 270.

### Ejercicio 1: Simulación de Montecarlo y Visualización

In [ ]:
h = Handler(N_ITERS = 1)
results ,planes = h.simulate()

Simulation:: 100%|██████████| 1/1 [00:00<00:00, 43.02it/s]


In [3]:
histories = [p for p in planes[0]["plane_history"]]
visualize.visualize(histories,2.5, 18*60, AIRCRAFT_SVG_PATH="plane.svg")

TypeError: 'NoneType' object is not subscriptable

### Ejercicio 2: Simulación con promedio de 1 arribo por hora
Si el promedio de arribos es 1 avión por hora y nosotros medimos minuto a minuto, $\lambda=\frac{1}{60}$

In [ ]:
ej2 = Handler(N_ITERS = 10_000, LAMBDA = 1/60)
results, simulation_history = ej2.simulate()

### Ejercicio 3: Probabilidad de que lleguen 5 aviones en una hora

In [ ]:
N_ITERS = 10_000
LAMBDA = 1/60
props_five = np.zeros(N_ITERS)
for i, sim in enumerate(plane_history):
    first_appearances = [sim["plane_history"][i][0][0] for i in range(len(sim["plane_history"]))]
    groups = np.zeros((18,))
    more_than_five = 0
    for t0 in first_appearances:
        min_to_hour = int(t0 // 60)
        groups[min_to_hour] += 1
    more_than_five += (groups >= 5).astype(int).sum() / 18
    props_five[i] = more_than_five

print("Proba de que lleguen más de 5 aviones en una hora:", props_five.mean())
print("Error de estimación:", props_five.std())

Podríamos calcularlo de forma analítica de la siguiente manera:

$$
\begin{aligned}
    \text{Sea } X &= 
    \begin{cases}
        1 & \text{si arribó un avión en ese minuto} \\
        0 & \text{si no}
    \end{cases} \\[6pt]
    &X \sim \text{Be}\!\left(\tfrac{1}{60}\right) \\[6pt]
    \text{Definimos } S &= \sum_{i=1}^{60} X_i \sim Bin(60, \frac{1}{60}) \\[6pt]
    \text{Luego} \ Y &= "\text{proba de que lleguen 5 aviones en una hora}" \ = \ P(S \ge 5) \\[6pt]
    \text{Así, podemos calcular analíticamente: } P(S \ge 5) &= 1 - P(S \lt 5)
\end{aligned}
$$

In [ ]:
rv = binom(60, 1/60)
1 - rv.cdf(4)

Así, vemos que el valor teórico de $P(S \ge 5)$  es 0.00328 $\approx$ 0.0033, lo cual no está lejos del valor estimado

### Ejercicio 4: Simulación de arribos para distintas tasas de arribo

In [ ]:
lambdas = [0.02, 0.1 , 0.2, 0.5, 1]
results = []

for l in lambdas:
    h = Handler(N_ITERS=10_000, LAMBDA= l)
    res, p_hist = h.simulate()
    results.append({
        "lambda": l,
        "numeric_results": res,
        "plane_history": p_hist
    })
# Metricas a mostrar aca:
# - Media de congestion por avion. Media de atraso en el arribo esperado (sta - ata)
# - Cantidad de aviones que llegaron en funcion de lambda
# - Cantidad de desvios a Montevideo
# - Cantidad de veces que un avion debe reposicionarse
# Agregar el error para cada una de estas variables

### Ejercicio 5: Simulación con interrupción de aterrizaje

In [ ]:
lambdas = [0.02, 0.1 , 0.2, 0.5, 1]
results = []
bounce_prob = 0.1
for l in lambdas:
    h = Handler(LAMBDA=l, PROP_BOUNCE=bounce_prob)
    res, p_hist = h.simulate()
    results.append({
        "lambda": l,
        "numeric_results": res,
        "plane_history": p_hist
    })
# Metricas a mostrar aca:
# - Media de congestion por avion. Media de atraso en el arribo esperado (sta - ata)
# - Cantidad de aviones que llegaron en funcion de lambda
# - Cantidad de desvios a Montevideo
# - Cantidad de veces que un avion debe reposicionarse
# Agregar el error para cada una de estas variables

### Ejercicio 6: Simulación con cierre de media hora

In [ ]:
arrival_rate = 10 / 60
h = Handler(LAMBDA=arrival_rate, CLOSING_TIME=240) # closing_time = 240 -> el aeropuerto cierra a las 10:00 AM
res, p_hist = h.simulate()
# Metricas a mostrar aca:
# - Media de congestion por avion. Media de atraso en el arribo esperado (sta - ata)
# - Cantidad de aviones que llegaron en funcion de lambda
# - Cantidad de desvios a Montevideo
# - Cantidad de veces que un avion debe reposicionarse
# Agregar el error para cada una de estas variables

### Ejercicio 7: Cambios para mejorar el funcionamiento del aeropuerto

Para la siguiente experimentación y cambios de políticas vamos a usar $\lambda = 0.1$ que representa el valor que estimamos en función de los datos empíricos obtenidos de aerolíneas y otros portales de aviación.